In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when


In [2]:
# Criar a sessão Spark (criar apenas uma vez)
spark = SparkSession.builder.appName("VendasFinalETL").getOrCreate()

In [3]:
# ler o arquivo transformado de vendas
# Especifica o caminho do arquivo
caminho_vendas_tratadas = "C:\\Users\\Theuzao\\Desktop\\panvel_datalake\\data\\transformed\\vendas_transformed"
vendas = spark.read.csv(caminho_vendas_tratadas, header=True, inferSchema=True)


In [4]:
# ler o arquivo transformado de itens vendas
# Especifica o caminho do arquivo
caminho_itens_vendas = "C:\\Users\\Theuzao\\Desktop\\panvel_datalake\\data\\transformed\\itens_vendas"
itens_vendas = spark.read.csv(caminho_itens_vendas, header=True, inferSchema=True)


In [5]:
# ler o arquivo transformado de clientes_opt
# Especifica o caminho do arquivo
caminho_clientes_opt = "C:\\Users\\Theuzao\\Desktop\\panvel_datalake\\data\\transformed\\clientes_opt"
clientes_opt = spark.read.csv(caminho_clientes_opt, header=True, inferSchema=True)


In [6]:
# ler o arquivo transformado de itens clientes
# Especifica o caminho do arquivo
caminho_clientes = "C:\\Users\\Theuzao\\Desktop\\panvel_datalake\\data\\transformed\\clientes"
clientes = spark.read.csv(caminho_clientes, header=True, inferSchema=True)

In [7]:
# join entre os dataframes vendas e itens_vendas utilizando n_id_vd_fil e n_id_fil como parametros
df_vendas_itens = vendas.join(
    itens_vendas,
    (vendas["n_id_vd_fil"] == itens_vendas["n_id_vd_fil"]) & 
    (vendas["n_id_fil"] == itens_vendas["n_id_fil"]),
    "inner"
)


In [8]:
# join entre os dataframes df_vendas_itens e clientes utilizando o v_cli_cod com v_id_cli
df_vendas_clientes = df_vendas_itens.join(
    clientes,
    df_vendas_itens["v_cli_cod"] == clientes["v_id_cli"],
    "inner"
)


In [9]:
#join entre df_vendas_clientes e clientes_opt utilizando o v_id_cli
df_vendas_final = df_vendas_clientes.join(
    clientes_opt,
    df_vendas_clientes["v_id_cli"] == clientes_opt["v_id_cli"],
    "left"
)


In [10]:
#define atraves do when o valor da coluna com base na preferencia do cliente
df_vendas_final = df_vendas_final.withColumn(
    "canal_venda",
    when(df_vendas_final["b_email"] == "Sim", "Site")
    .when(df_vendas_final["b_push"] == "Sim", "App")
    .otherwise("Loja")
)


In [11]:
#renomeia as colunas
df_vendas_final = df_vendas_final.select(
    vendas["n_id_fil"].alias("codigo_filial"),
    vendas["n_id_vd_fil"].alias("codigo_cupom_venda"),
    vendas["d_dt_vd"].alias("data_emissao"),
    itens_vendas["n_id_it"].alias("codigo_item"),
    itens_vendas["n_vlr_vd"].alias("valor_unitario"),
    itens_vendas["n_qtd"].alias("quantidade"),
    vendas["v_cli_cod"].alias("codigo_cliente"),
    itens_vendas["v_it_vd_conv"].alias("tipo_desconto"),
    df_vendas_final["canal_venda"]
)


In [12]:
#mostra os dados de vendas_final
df_vendas_final.show()


+-------------+------------------+-------------------+-----------+--------------+----------+--------------------+-------------+-----------+
|codigo_filial|codigo_cupom_venda|       data_emissao|codigo_item|valor_unitario|quantidade|      codigo_cliente|tipo_desconto|canal_venda|
+-------------+------------------+-------------------+-----------+--------------+----------+--------------------+-------------+-----------+
|      2701384|            401731|2023-07-06 21:00:00|  454248626|         15.99|       1.0|031FF300BFC27BFCD023|          NAO|       Site|
|      2701384|            401731|2023-07-06 21:00:00|  504129626|         23.99|       1.0|031FF300BFC27BFCD023|          NAO|       Site|
|      2701384|            401731|2023-07-06 21:00:00|  403349206|         17.59|       1.0|031FF300BFC27BFCD023|          NAO|       Site|
|      2701384|            401731|2023-07-06 21:00:00|  451479626|         15.59|       1.0|031FF300BFC27BFCD023|          NAO|       Site|
|      2701384|     

In [13]:
# Salvar o DataFrame em formato Parquet
df_vendas_final.write.mode("overwrite").parquet("C:/Users/Theuzao/Desktop/panvel_datalake/data/processed/vendas_parquet")


In [14]:
#Encerra sessão Spark
spark.stop()